In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
N_batch = 32
N_block = 8
max_iters = 3000
eval_interval = 300
lr = 1e-3
eval_iters = 200

In [3]:
torch.manual_seed(1337)

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [12]:
chars = sorted(list(set(text)))
N_vocab = len(chars)

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s : [ stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [13]:
data = torch.tensor(encode(text), dtype = torch.long)
N = int(0.9 * len(data))
train_data = data[:N]
valid_data = data[N:]

In [14]:
def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - N_block, (N_batch, ))
    x = torch.stack([data[i:i+N_block] for i in ix])
    y = torch.stack([data[i+1:i+N_block+1] for i in ix])
    return x, y

In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class BigramLanguageModel(nn.Module):

    def __init__(self, N_vocab):
        super().__init__()
        self.token_embedding_table = nn.Embedding(N_vocab, N_vocab)

    def forward(self, idx, targets = None):

        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [17]:
model = BigramLanguageModel(N_vocab)
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [21]:
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        #print(f'Step {iter:%5d} | train loss {losses['train']:.4f} | cal loss {losses['val']:.4f}')

    xb, yb  = get_batch('train')

    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    

/home/dsladmin/anaconda3/lib/python3.11/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [19]:
losses['train']

NameError: name 'losses' is not defined

In [22]:
context = torch.zeros((1,1), dtype = torch.long)
print(decode(model.generate(context, max_new_tokens = 500)[0].tolist()))


R$Vicouant ngFJgrkl stMaATu.Z3?
IS:vGr h
NJ;bd e fr WI:Vojulis.f mirowkery of?H
ig' hthbercqZ.NIat.LHUwed.; wn,!
RHnpImilapr ar'astoxo towstxLaJY h

BSngal der lled
S t
Lis; wyou, QVin.'sba$; evJVTe b;odt de llavJ;PEDugr w$JVI ft s
BEO:h we, zH:dallAIUh Vomtat kRMvin!
GMJces, tu tJpe;owmutto t adsweJee s scS:

to:-SHAENTha:

Be War pETmy valemCy?acoetew stWw-h.
THTuSPTI;''WIim, ar ar se, swfHAd ner p;prve.
AMORof thohOGBr tLlaigl madoncte onAJeyobl j, ng!e trdser'zXI ee QozWAP:ferve y thinstuaw

